In [1]:
import pandas as pd
import numpy as np
from snownlp import sentiment, SnowNLP
import jieba

In [2]:
from aip import AipNlp

"""APPID AK SK """
APP_ID = '19110692'
API_KEY = 'hRfsROHtraDKnffdXSeN3ADG'
SECRET_KEY = 'jdiPXp3pLVi6S4utmGvu6IG9Gk6oxu3w'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [87]:
# prepare stopwords as a list
stop_words = jieba.load_userdict("stop_words")
stopwords=pd.read_csv('stop_words',index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values
stopwords = stopwords.tolist()

In [178]:
incident = pd.read_csv("Comments_Dataset_Mar/trump.csv")
# incident=incident[:601]

In [179]:
columns = ['博文网址','发帖子人','博文发布时间','评论数','来自','转发数','点赞数','博文内容','博文图片','博文视频','评论者','二级评论人','二级评论时间','二级评论内容']
incident.drop(columns, inplace=True, axis=1)
incident.columns= ['comment_content','comment_time','no_of_likes']
incident.fillna(value='无',inplace=True)

In [180]:
incident.describe()

,comment_content,comment_time,no_of_likes
count,233,233,233
unique,211,152,149
top,图片评论¡评论配图,3月20日 07:28,无
freq,5,6,15


In [96]:
def remove_stopwords(sentence):

    segList = jieba.cut(sentence)
    segResult = []
    for w in segList:
        segResult.append(w)

    newSent = []
    for word in segResult:
        if word in stopwords:
            # print "stopword: %s" % word
            continue
        else:
            newSent.append(word)
    return newSent

In [35]:
# clean_comments = []
# for comment in incident['comment_content']:
#     string = ''
#     for word in remove_stopwords(comment):
#         string += word
#     clean_comments.append(string)
# incident['comments'] = clean_comments
# # clean_comments

In [172]:
incident.head(5)

,comment_content,comment_time,no_of_likes
0,同意，中国人回来没问题，外国友人这会儿就先别过来了。,3月26日 23:34,1291
1,今天27号出永居结果，要是废除这个新条例就更好了,3月27日 00:43,231
2,我终于看到这个信息了。就该对非本国籍人员进行统一管理,3月26日 23:34,803
3,这是我今天看到最高兴的事了,3月26日 23:35,18
4,主要是外国人太能瞎混了，严重不靠谱，实在招架不住了,3月26日 23:43,575


In [157]:
# # update sentiments dictionary and train it 
# from snownlp import sentiment
# sentiment.train('neg.txt', 'pos.txt')
# sentiment.save('sentiment.marshal')

In [12]:
# sentimentslist= []
# for i in incident['comment_content'][:10]:
#     s = SnowNLP(i)
#     sentimentslist.append(s.sentiments)
    
#     result = []
#     i = 0
#     while i<len(sentimentslist):
#         result.append(sentimentslist[i]-0.5)
#         i = i + 1
# result

In [181]:
sentimentslist = []
sentimentsClass= []
for comment in incident['comment_content']:
    try:
        reply=client.sentimentClassify(comment)['items']
        sentimentslist.append(max(reply[0]['positive_prob'],reply[0]['negative_prob']))
        sentimentsClass.append(reply[0]['sentiment'])
    except:
        sentimentslist.append('NaN')
        sentimentsClass.append('NaN')
        pass
incident['sentiment_probability'] = sentimentslist
incident['sentiments_class'] = sentimentsClass
incident.head()
# sentimentsClass

,comment_content,comment_time,no_of_likes,sentiment_probability,sentiments_class
0,确实，没有人比他更懂新冠了！,3月19日 23:01,177672,0.554862,0
1,确实，没有人比他更懂新冠了！,3月19日 23:01,177672,0.554862,0
2,2019年6月美国出现病毒肺炎死亡200多人的病例，症状同是发热咳嗽和肺部变白，特朗普说是电...,3月20日 07:38,24428,0.999664,0
3,他投的毒，他肯定最懂了,3月19日 23:18,18524,0.746926,0
4,愿病毒早日战胜特朗普,3月20日 07:46,12795,0.515194,1


In [20]:
# incident.to_csv(r'leave_wuhan_sentiments.csv', index = False,encoding='utf-8')

In [21]:
# del incident['sentiments_score']

In [175]:
# incident.head(20)

In [182]:
total = incident['comment_content'].count()

In [183]:
num_list = incident['sentiments_class'].value_counts(sort=False).tolist()
percent_list = []
for num in num_list:
    percent_list.append(round(num/total*100,4))
# econs_incident['sentiments_class'].value_counts(sort=False)
percent_list=percent_list[:-1]
incident['sentiments_class'].value_counts(sort=False)
percent_list

[64.8069, 4.2918, 24.8927]

In [101]:
def pie_base():
    from pyecharts import options as opts
    from pyecharts.charts import Page, Pie
    v1=["negative","neutral","positive"]
    v2=percent_list
    
    c = (
        Pie()
        .add("", [list(z) for z in zip(v1,v2)])
        .set_global_opts(title_opts=opts.TitleOpts(title="sentiment percentage"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}%"))
    )
    return c
pie_base().render_notebook()

In [16]:
pie_base().render('incident_sentiment_percent/school_open_sentiment.html')

'C:\\Users\\Zhengnan\\IS434 Project\\incident_sentiment_percent\\school_open_sentiment.html'